In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
def linear_HRV_Classification_Tree_Algorithm(lf, pnn50):
    if (lf<899.58 and pnn50>.9873) or (lf<277.28 and pnn50<.9873):
        return 'stress'
    if lf>899.58 or (lf>277.28 and pnn50<0.9783):
        return 'no stress'
    return 'uncertain state'

# Any results you write to the current directory are saved as output.
"""
from sklearn import tree
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
test = pd.read_csv('../input/swell-heart-rate-variability-hrv/hrv dataset/hrv dataset/data/final/test.csv')
train = pd.read_csv('../input/swell-heart-rate-variability-hrv/hrv dataset/hrv dataset/data/final/train.csv')
test.head(3)
right = 0;
wrong = 0;
for index, row in test.head(len(test.index)).iterrows():
     # access data using column names
     resultStr = linear_HRV_Classification_Tree_Algorithm(row['LF'], row['pNN50'])
     if resultStr == 'no stress' and row['condition'] == 'no stress':
        right+=1
     elif resultStr == 'stress' and row['condition'] != 'no stress':
        right+=1
     else:
        wrong+=1
print(right)
print(wrong)
print(right/(right+wrong))
"""

'''
import os
import sklearn.pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
#NOTE: This code is just a quick and dirty proof of concept. Our implimentation in the paper is completely different

def load_train_set():
    #Loading a hdf5 file is much much faster
    return pd.read_csv('../input/swell-heart-rate-variability-hrv/hrv dataset/hrv dataset/data/final/train.csv')
def load_test_set():
    #Loading a hdf5 file is much much faster
    return pd.read_csv('../input/swell-heart-rate-variability-hrv/hrv dataset/hrv dataset/data/final/train.csv')

def simple_model_evaluation():
    select = SelectKBest(k=20)
    train =load_train_set()
    test = load_test_set()
    target = 'condition'
    hrv_features = list(train)
    hrv_features = [x for x in hrv_features if x not in [target]]
    X_train= train[hrv_features]
    y_train= train[target]
    X_test = test[hrv_features]
    y_test = test[target]
    classifiers = [
                    DecisionTreeClassifier(),
                    RandomForestClassifier(n_estimators=100, max_features='log2', n_jobs=-1),
                    SVC(C=20, kernel='rbf'),   
                 ]
    for clf in classifiers:
        name = str(clf).split('(')[0]
        if 'SVC' == name:
            # Normalize the attribute values to mean=0 and variance=1
            from sklearn.preprocessing import StandardScaler
            scaler = StandardScaler()
            scaler.fit(X_train)
            X_train = scaler.transform(X_train)
            X_test = scaler.transform(X_test)
        clf = RandomForestClassifier()
        steps = [('feature_selection', select),
             ('model', clf)]
        pipeline = sklearn.pipeline.Pipeline(steps)
        pipeline.fit(X_train, y_train)
        y_prediction = pipeline.predict(X_test)
        print("----------------------------{0}---------------------------".format(name))
        print(sklearn.metrics.classification_report(y_test, y_prediction))
        print()
        print()
        
simple_model_evaluation()
'''
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV 
from tpot import TPOTClassifier
%matplotlib inline
import matplotlib.pyplot as plt
from scipy import signal 
import pickle


import sklearn.metrics
from sklearn.model_selection import cross_val_score
from sklearn import svm
import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support 

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)
dataframe_hrv = pd.read_csv("../input/swell-heart-rate-variability-hrv/hrv dataset/hrv dataset/data/final/train.csv")
dataframe_hrv = dataframe_hrv.reset_index(drop=True)
display(dataframe_hrv.head(5))

/kaggle/input/swell-heart-rate-variability-hrv/hrv dataset/quick_start.py
/kaggle/input/swell-heart-rate-variability-hrv/hrv dataset/hrv dataset/quick_start.py
/kaggle/input/swell-heart-rate-variability-hrv/hrv dataset/hrv dataset/papers/Koldijk et. al - 2018.pdf
/kaggle/input/swell-heart-rate-variability-hrv/hrv dataset/hrv dataset/papers/Kizito et. al - 2019.pdf
/kaggle/input/swell-heart-rate-variability-hrv/hrv dataset/hrv dataset/papers/Koldijk et al. - 2014pdf.pdf
/kaggle/input/swell-heart-rate-variability-hrv/hrv dataset/hrv dataset/data/final/train.csv
/kaggle/input/swell-heart-rate-variability-hrv/hrv dataset/hrv dataset/data/final/test.csv
/kaggle/input/swell-heart-rate-variability-hrv/hrv dataset/hrv dataset/data/raw/labels/hrv stress labels.xlsx
/kaggle/input/swell-heart-rate-variability-hrv/hrv dataset/hrv dataset/data/raw/rri/p18.txt
/kaggle/input/swell-heart-rate-variability-hrv/hrv dataset/hrv dataset/data/raw/rri/p22.txt
/kaggle/input/swell-heart-rate-variability-hrv/hr

,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,HR,pNN25,pNN50,SD1,SD2,KURT,SKEW,MEAN_REL_RR,MEDIAN_REL_RR,SDRR_REL_RR,RMSSD_REL_RR,SDSD_REL_RR,SDRR_RMSSD_REL_RR,KURT_REL_RR,SKEW_REL_RR,VLF,VLF_PCT,LF,LF_PCT,LF_NU,HF,HF_PCT,HF_NU,TP,LF_HF,HF_LF,sampen,higuci,datasetId,condition
0,885.157845,853.763730,140.972741,15.554505,15.553371,9.063146,69.499952,11.133333,0.533333,11.001565,199.061782,-0.856554,0.335218,-0.000203,-0.000179,0.017080,0.007969,0.007969,2.143342,-0.856554,0.335218,2661.894136,72.203287,1009.249419,27.375666,98.485263,15.522603,0.421047,1.514737,3686.666157,65.018055,0.015380,2.139754,1.163485,2,no stress
1,939.425371,948.357865,81.317742,12.964439,12.964195,6.272369,64.363150,5.600000,0.000000,9.170129,114.634458,-0.408190,-0.155286,-0.000059,0.000611,0.013978,0.004769,0.004769,2.930855,-0.408190,-0.155286,2314.265450,76.975728,690.113275,22.954139,99.695397,2.108525,0.070133,0.304603,3006.487251,327.296635,0.003055,2.174499,1.084711,2,interruption
2,898.186047,907.006860,84.497236,16.305279,16.305274,5.182201,67.450066,13.066667,0.200000,11.533417,118.939253,0.351789,-0.656813,-0.000011,-0.000263,0.018539,0.008716,0.008716,2.127053,0.351789,-0.656813,1373.887112,51.152225,1298.222619,48.335104,98.950472,13.769729,0.512671,1.049528,2685.879461,94.280910,0.010607,2.135350,1.176315,2,interruption
3,881.757865,893.460030,90.370537,15.720468,15.720068,5.748591,68.809562,11.800000,0.133333,11.119476,127.318597,-0.504947,-0.386138,0.000112,0.000494,0.017761,0.008660,0.008660,2.050988,-0.504947,-0.386138,2410.357408,70.180308,1005.981659,29.290305,98.224706,18.181913,0.529387,1.775294,3434.520980,55.328701,0.018074,2.178341,1.179688,2,no stress
4,809.625331,811.184865,62.766242,19.213819,19.213657,3.266724,74.565728,20.200000,0.200000,13.590641,87.718281,-0.548408,-0.154252,-0.000100,-0.002736,0.023715,0.013055,0.013055,1.816544,-0.548408,-0.154252,1151.177330,43.918366,1421.782051,54.242160,96.720007,48.215822,1.839473,3.279993,2621.175204,29.487873,0.033912,2.221121,1.249612,2,no stress


In [2]:
selected_x_columns = ['HR', 'RMSSD', 'pNN50', 'TP', 'VLF', 'LF', 'HF','LF_HF']
X = dataframe_hrv[selected_x_columns]
display(X.head(5))

,HR,RMSSD,pNN50,TP,VLF,LF,HF,LF_HF
0,69.499952,15.554505,0.533333,3686.666157,2661.894136,1009.249419,15.522603,65.018055
1,64.363150,12.964439,0.000000,3006.487251,2314.265450,690.113275,2.108525,327.296635
2,67.450066,16.305279,0.200000,2685.879461,1373.887112,1298.222619,13.769729,94.280910
3,68.809562,15.720468,0.133333,3434.520980,2410.357408,1005.981659,18.181913,55.328701
4,74.565728,19.213819,0.200000,2621.175204,1151.177330,1421.782051,48.215822,29.487873


In [3]:
def fix_stress_labels(df='',label_column='condition'):
    df['condition'] = np.where(df['condition']=='no stress', 0, 1)
    display(df["condition"].unique())
    return df
dataframe_hrv = fix_stress_labels(df=dataframe_hrv)
Y = dataframe_hrv['condition']
display(Y.head(5))

array([0, 1])

0    0
1    1
2    1
3    0
4    0
Name: condition, dtype: int64

In [4]:
def do_tpot(generations=5, population_size=10,X='',y=''):

    X_train, X_test, y_train, y_test = train_test_split(X, y,train_size=0.80,test_size=0.20)
    tpot = TPOTClassifier(generations=generations, population_size=population_size, verbosity=2,cv=3)
    tpot.fit(X_train, y_train)
    print(tpot.score(X_test, y_test))
    tpot.export('tpot_pipeline.py')
    return tpot

tpot_classifer = do_tpot(generations=5, population_size=20,X=X,y=Y)

Generation 1 - Current best internal CV score: 0.9989066828578618
Generation 2 - Current best internal CV score: 0.99942795468731
Generation 3 - Current best internal CV score: 0.99942795468731
Generation 4 - Current best internal CV score: 0.9995430407818705
Generation 5 - Current best internal CV score: 0.9998206013628718

Best pipeline: ExtraTreesClassifier(input_matrix, bootstrap=True, criterion=entropy, max_features=1.0, min_samples_leaf=1, min_samples_split=16, n_estimators=100)
0.9999864605052939


In [5]:
newDataframe_hrv = pd.read_csv("../input/swell-heart-rate-variability-hrv/hrv dataset/hrv dataset/data/final/test.csv")
newdataframe_hrv = dataframe_hrv.reset_index(drop=True)
new_selected_x_columns = ['HR', 'RMSSD', 'pNN50', 'TP', 'VLF', 'LF', 'HF','LF_HF']
newX = newDataframe_hrv[selected_x_columns]
display(newX.head(5))

,HR,RMSSD,pNN50,TP,VLF,LF,HF,LF_HF
0,84.121868,12.361264,0.000000,1698.605390,1016.073759,615.914573,66.617057,9.245599
1,71.478642,19.298880,0.200000,2358.884694,765.518473,1566.866135,26.500086,59.126832
2,63.874293,21.342715,1.800000,4328.633724,2237.739905,2074.868884,16.024935,129.477524
3,74.330531,11.771814,0.533333,2854.449091,2330.980957,505.886664,17.581470,28.773854
4,82.092049,13.357748,0.666667,5310.027472,4750.624447,524.203971,35.199054,14.892559


In [6]:
pred = tpot_classifer.predict_proba(newX)
dfpred = pd.DataFrame(pred)
display(dfpred.head(5))

,0,1
0,1.000000,0.000000
1,0.000000,1.000000
2,1.000000,0.000000
3,0.994048,0.005952
4,0.126046,0.873954


In [7]:
display(newDataframe_hrv['condition'].head(5))

0    no stress    
1    time pressure
2    no stress    
3    no stress    
4    interruption 
Name: condition, dtype: object

In [8]:
export_csv1 = dfpred.to_csv ('predicted_stress.csv', index = None, header=True)
export_csv2 = newDataframe_hrv['condition'].to_csv('actual_condition.csv', index = None, header=True)